## MEWpy
Metabolic Engineering Workbench

Gecko model simulation example

In [1]:
from mewpy.model.gecko import GeckoModel
model = GeckoModel('single-pool')

The Simulation class enables to easily perform simulations considering specific environmental conditions as well as other constraint, such protein knockouts or over/under expression. 
 

In [2]:
from mewpy.simulation import get_simulator
envcond = {'r_1714_REV':(0,1000)}
simul = get_simulator(model,envcond= envcond)

In [3]:
# FBA
result =simul.simulate()
print(result)

# Other methods can explicitly be defined
from mewpy.simulation import  SimulationMethod
result= simul.simulate(method= SimulationMethod.pFBA)
reference = result.fluxes


objective: 0.3254251070535827
Status: SStatus.OPTIMAL


In [4]:
# Essential proteins
simul.essential_proteins[:10]

['draw_prot_P49367',
 'draw_prot_P17423',
 'draw_prot_Q12189',
 'draw_prot_P15625',
 'draw_prot_P07277',
 'draw_prot_P00958',
 'draw_prot_P07172',
 'draw_prot_P09950',
 'draw_prot_P04802',
 'draw_prot_P53852']

In [21]:
constraints ={'draw_prot_Q04396': (2.3029771980628338e-07,1000),'draw_prot_P27515':(0,0.0),'draw_prot_P53687':(0.0,1000),'draw_prot_P81450': 0,
              'draw_prot_P41939': (3.944331158229099e-06,1000),'r_0659_REVNo1':0,'draw_prot_P19657':0,'draw_prot_P37254':(0.0,1000),
              'draw_prot_P33317': 0,'draw_prot_P54114':(0,0.0),'draw_prot_P25373':(0.0,1000),'draw_prot_P08067':(0,2.1621430284827036e-06),
              'draw_prot_P39002': (0.0,1000),'draw_prot_Q04792':(0.0,1000),'draw_prot_Q12320':(0.0,1000),'draw_prot_P07347':(0.0,1000),
              'draw_prot_P00958':(0,3.4862363761891556e-06),'draw_prot_P15179': 0, 'draw_prot_P53332':(0.0,1000),'draw_prot_P23202':(0.0,1000),
              'draw_prot_P21672':0,'draw_prot_P32191':0,'draw_prot_P38071': 0,'draw_prot_P06169':(0.00019588347980199452,1000),'draw_prot_P22803':0,
              'draw_prot_P38986':(0.0,1000),'draw_prot_P32621': (0.0,1000),'draw_prot_P27472':(1.7311263946838522e-07,1000),
              'draw_prot_P00549': (0,1.624598159388334e-05),'draw_prot_P32796':0}

In [24]:
result = simul.simulate(constraints=constraints)
result.fluxes['r_2111']
result.fluxes['r_2056']

4.651395466320041

## GECKO model Optimization

The optimization is performed by defining an Evolutionary Algorithm Optimization Problem.
MEWpy can resort to inspyred as well as to jmetalpy packages to drive the optimization process.

To define a problem we start by identifiying the objective functions:

In [28]:
from mewpy.optimization.evaluation import BPCY,WYIELD

evaluator_1 = WYIELD("r_2111", "r_1913")
evaluator_2 = BPCY("r_2111", "r_1913", uptake = "r_1714_REV", method=SimulationMethod.lMOMA ,reference=reference) 

In [31]:
# creates a new KO optimization problem
from mewpy.problems import GeckoKOProblem

# The problem takes as parameters the model, the objectives, optional environmental conditions,
# and the maximum number of simultaneous KOs. Other parameters can be passed such as, for example, 
# target or, alternatively, non target proteins. If not provided, targets are all non essential proteins. 

problem = GeckoKOProblem(model, 
                          fevaluation=[evaluator_1,evaluator_2], 
                          envcond = envcond,
                          candidate_max_size = 30)

In [34]:
from mewpy.optimization import EA

# A new instance of the EA optimizer for the problem with a stopping criteria of 10 iterations (1000 evaluations)
ea = EA(problem, max_generations = 10, mp =True, visualizer = True)

# runs the optimization
final_pop = ea.run()


Generation Evaluation|      Worst       Best     Median    Average    Std Dev|      Worst       Best     Median    Average    Std Dev|
         0        100|   0.000000   0.000017   0.000000   0.000001   0.000005|   0.000000   0.000000   0.000000   0.000000   0.000000|
         1        200|   0.000000   0.000017   0.000000   0.000002   0.000006|   0.000000   0.000000   0.000000   0.000000   0.000000|
         2        300|   0.000000   0.000017   0.000000   0.000004   0.000007|   0.000000   0.000000   0.000000   0.000000   0.000000|
         3        400|   0.000000   0.000017   0.000000   0.000008   0.000008|   0.000000   0.000000   0.000000   0.000000   0.000000|
         4        500|   0.000000   0.000017   0.000016   0.000013   0.000006|   0.000000   0.000000   0.000000   0.000000   0.000000|
         5        600|   0.000016   0.000017   0.000017   0.000017   0.000000|   0.000000   0.000000   0.000000   0.000000   0.000000|
         6        700|   0.000016   0.000017   0.000017

In [ ]:
# Save the solutions to a csv file
import mewpy.utils.utilities as util
util.population_to_csv(problem,final_pop,'results.csv')